In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#Import Libraries 
import tensorflow as tf 
import keras
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import pandas as pd 
from PIL import Image
import glob

In [3]:
#Define Path 
train_dir = '../input/plant-seedlings-classification/train'
test_dir = '../input/plant-seedlings-classification/test'

# train_dr = os.listdir(train_dir)
# test_dr = os.listdir(test_dir)

In [4]:
# #define parameters 
# batch_size = 32
# img_width = 240
# img_height = 240 
# epochs = 20

#For inceptionresnetv2
batch_size = 32
img_width = 299
img_height = 299
epochs = 20

In [5]:
#Split data 

train_image_data = ImageDataGenerator(
    validation_split = 0.2,
    rescale = 1.0/255.0,
    horizontal_flip = True, 
    zoom_range=0.2, 
    shear_range = 0.2,
)

train_ds = train_image_data.flow_from_directory(
  train_dir,
  shuffle = True,
  target_size=(img_height, img_width),
  class_mode = 'categorical',
  batch_size=batch_size,
  subset="training")

val_ds = train_image_data.flow_from_directory(
  train_dir,
  shuffle = True,
  target_size=(img_height, img_width),
  class_mode = 'categorical',
  batch_size=batch_size,
  subset="validation")

In [6]:
train_steps = train_ds.samples//batch_size
val_steps = val_ds.samples//batch_size

In [7]:
#Define model 
#Model 2
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
model = tf.keras.Sequential([
            InceptionResNetV2(input_shape=[299,299, 3], include_top=False, weights='imagenet'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(12, activation='softmax')
])
model.summary()

In [8]:
model.compile(optimizer= 'sgd', loss='categorical_crossentropy', metrics=['accuracy'])

#Save model checkpoint
checkpoint_filepath = './kaggle/working/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [9]:
#Train Model
history = model.fit_generator(
                train_ds,
                steps_per_epoch = train_steps,
                validation_data = val_ds, 
                validation_steps = val_steps,
                epochs = epochs)

In [10]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [12]:
#Apply model to test set 
#Initialise test generator
import shutil
import pathlib
import os
# !rm -rf 'kaggle/working/new_test'

# test_data = pathlib.Path(test_dir)
# #new_test_path = pathlib.Path('kaggle/working/new_test')
#shutil.move(test_dir, 'kaggle/working/')

new_test_path = '/kaggle/working/test'
#shutil.move(test_dir, new_test_path)

test_image_data = ImageDataGenerator(rescale=1./255)
test_ds = test_image_data.flow_from_directory('/kaggle/working/',
                                              target_size=(img_height,img_width),
                                              batch_size = 1,
                                              class_mode ='categorical',
                                              shuffle = False)

In [13]:
model.evaluate_generator(generator=val_ds, steps=val_steps)

In [14]:
test_ds.reset() #maintain test order
test_predictions = model.predict_generator(test_ds, test_ds.samples, verbose=1 )

In [15]:
import numpy as np
predicted_plant_type = np.argmax(test_predictions, axis=1)

In [16]:
labels = (train_ds.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_plant_type]

In [18]:
test_pred = []
filenames=test_ds.filenames
for name in range(len(filenames)):
    test_pred.append(filenames[name].split('/')[1])

results=pd.DataFrame({"file":test_pred,
                      "species":predictions})
results.to_csv("submission.csv",index=False)